### 事务
+ MySQL事务主要用于处理操作量大，复杂度高的数据。它是一个操作序列，这些操作要么都执行，要么都不执行，它是一个不可分割的工作单位。<br><br>
+ 在MySQL中只有使用了Innodb数据库引擎的数据库或表才支持事务。<br><br>
+ 事务处理可以用来维护数据库的完整性，保证成批的SQL语句要么全部执行，要么全部不执行。<br><br>
+ 事务用来管理 insert,update,delete 语句

#### 事务四大特性（简称ACID）
+ 原子性（Atomicity，或称不可分割性）
+ 一致性（Consistency）
+ 隔离性（Isolation，又称独立性）
+ 持久性（Durability）。<br><br>

+ 原子性：一个事务（transaction）中的所有操作，要么全部完成，要么全部不完成，不会结束在中间某个环节。事务在执行过程中发生错误，会被回滚（Rollback）到事务开始前的状态，就像这个事务从来没有执行过一样。<br><br>

+ 一致性：在事务开始之前和事务结束以后，数据库的完整性没有被破坏。这表示写入的资料必须完全符合所有的预设规则，这包含资料的精确度、串联性以及后续数据库可以自发性地完成预定的工作。<br><br>

+ 隔离性：数据库允许多个并发事务同时对其数据进行读写和修改的能力，隔离性可以防止多个事务并发执行时由于交叉执行而导致数据的不一致。事务隔离分为不同级别，包括读未提交（Read uncommitted）、读提交（read committed）、可重复读（repeatable read）和串行化（Serializable）。一个事务所做的修改在最终提交以前，对其他事务是不可见的。<br><br>

+ 持久性：事务处理结束后，对数据的修改就是永久的，即便系统故障也不会丢失。

#### 以银行账户转账为例：

+ 1.START TRANSACTION;
+ 2.SELECT balance FROM checking WHERE customer_id = 10233276;
+ 3.UPDATE checking SET balance = balance - 200.00 WHERE customer_id = 10233276;
+ 4.UPDATE savings SET balance = balance + 200.00 WHERE customer_id = 10233276;
+ 5.COMMIT;<br><br>
+ 原子性：要么完全提交（10233276的checking余额减少200，savings 的余额增加200），要么完全回滚（两个表的余额都不发生变化）
+ 一致性：这个例子的一致性体现在200元不会因为数据库系统运行到第3行之后，第4行之前时崩溃而不翼而飞，因为事务还没有提交。
+ 隔离性：允许在一个事务中的操作语句会与其他事务的语句隔离开，比如事务A运行到第3行之后，第4行之前，此时事务B去查询checking余额时，它仍然能够看到在事务A中被减去的200元（账户钱不变），因为事务A和B是彼此隔离的。在事务A提交之前，事务B观察不到数据的改变。<br><br>
+ [注]:2,3,4这三步操作必须打包在一个事务中，任意一个步骤失败，必须回滚索引步骤。

#### mysql中的事务
+ mysql默认采用AutoCommit模式，也就是每个sql都是一个事务，并不需要显示的执行事务。如果autoCommit关闭，那么每个sql都默认开启一个事务，只有显式的执行“commit”后这个事务才会被提交。

### 事务的隔离级别
#### 并发事务带来的问题
+ 更新丢失（Lost Update）：当两个或多个事务选择同一行，然后基于最初选定的值更新该行时，由于每个事务都不知道其他事务的存在，就会发生丢失更新问题 －－最后一个事务的更新覆盖了由其他事务所做的更新。在一个事务提交之前，另一个事务不能访问同一行，则可避免此问题。<br><br>
+ 脏读（Dirty Reads）：一个事务正在对一条记录做修改，在这个事务完成并提交前，这条记录的数据就处于不一致状态； 这时，另一个事务也来读取同一条记录，如果不加控制，第二个事务读取了这些“脏”数据，并据此做进一步的处理，就会产生未提交的数据依赖关系。这种现象被形象地叫做"脏读"。<br><br>
+ 不可重复读（Non-Repeatable Reads）：一个事务在读取某些数据后的某个时间，再次读取以前读过的数据，却发现其读出的数据已经发生了改变、或某些记录已经被删除了！这种现象就叫做“不可重复读” 。<br><br>
+ 幻读（Phantom Reads）：一个事务按相同的查询条件重新读取以前检索过的数据，却发现其他事务插入了满足其查询条件的新数据，这种现象就称为“幻读” 。

#### 并发事务处理问题的解决办法
+ “更新丢失”通常是应该完全避免的。但防止更新丢失，并不能单靠数据库事务控制器来解决，需要应用程序对要更新的数据加必要的锁来解决，因此，防止更新丢失应该是应用的责任。
+ “脏读”、“不可重复读”和“幻读”，其实都是数据库读一致性问题，必须由数据库提供一定的事务隔离机制来解决：一种是加锁：在读取数据前，对其加锁，阻止其他事务对数据进行修改。另一种是数据多版本并发控制（MultiVersion Concurrency Control，简称 MVCC 或 MCC），也称为多版本数据库：不用加任何锁， 通过一定机制生成一个数据请求时间点的一致性数据快照（Snapshot)，并用这个快照来提供一定级别（语句级或事务级）的一致性读取。从用户的角度来看，好象是数据库可以提供同一数据的多个版本。

#### 4类隔离级别
##### 第1级别：Read Uncommitted(读取未提交内容)
+ 所有事务都可以看到其他未提交事务的执行结果
+ 本隔离级别很少用于实际应用，因为它的性能也不比其他级别好多少
+ 该级别引发的问题是——脏读(Dirty Read)：读取到了未提交的数据<br><br>
##### 第2级别：Read Committed(读取提交内容)
+ 这是大多数数据库系统的默认隔离级别（但不是MySQL默认的）
+ 它满足了隔离的简单定义：一个事务只能看见已经提交事务所做的改变
+ 这种隔离级别出现的问题是——不可重复读(Nonrepeatable Read)：不可重复读意味着我们在同一个事务中执行完全相同的select语句时可能看到不一样的结果。导致这种情况的原因可能有：有一个交叉的事务有新的commit，导致了数据的改变;一个数据库被多个实例操作时,同一事务的其他实例在该实例处理其间可能会有新的commit。<br><br>
##### 第3级别：Repeatable Read(可重读)
+ 这是MySQL的默认事务隔离级别
+ 它确保同一事务的多个实例在并发读取数据时，会看到同样的数据行
+ 此级别可能出现的问题——幻读(Phantom Read)：当用户读取某一范围的数据行时，另一个事务又在该范围内插入了新行，当用户再读取该范围的数据行时，会产生幻读。
+ InnoDB存储引擎通过多版本并发控制(MVCC，Multiversion Concurrency Control)机制解决幻读问题；InnoDB还通过间隙锁解决幻读问题<br><br>

##### 第4级别：Serializable(可串行化)
+ 这是最高的隔离级别
+ 它通过强制事务排序，使之不可能相互冲突，从而解决幻读问题。简言之,它是在每个读的数据行上加上共享锁。
+ 在这个级别，可能导致大量的超时现象和锁竞争<br><br>

#### 事务日志
+ 使用事务日志，存储引擎在修改表的数据时只需要修改其内存拷贝，再把该修改行为记录到持久在硬盘上的事务日志中，而不用每次都将修改的数据本身持久到磁盘。
+ 事务日志采用的是追加的方式，因此写日志的操作是磁盘上一小块区域内的顺序I/O，而不像随机I/O需要在磁盘的多个地方移动磁头，所以采用事务日志的方式相对来说要快得多。
+ 事务日志持久以后，内存中被修改的数据在后台可以慢慢刷回到磁盘。
+ 如果数据的修改已经记录到事务日志并持久化，但数据本身没有写回到磁盘，此时系统崩溃，存储引擎在重启时能够自动恢复这一部分修改的数据。
+ 目前来说，大多数存储引擎都是这样实现的，我们通常称之为预写式日志（Write-Ahead Logging），修改数据需要写两次磁盘。<br><br>

#### Mysql中的事务实现原理
+ 事务的实现是基于数据库的存储引擎。不同的存储引擎对事务的支持程度不一样。mysql中支持事务的存储引擎有innoDB和NDB。<br><br>
+ innoDB是mysql默认的存储引擎，默认的隔离级别是RR（Repeatable Read），并且在RR的隔离级别下更进一步，通过多版本并发控制（MVCC，Multiversion Concurrency Control）解决不可重复读问题，加上间隙锁（也就是并发控制）解决幻读问题。因此innoDB的RR隔离级别其实实现了串行化级别的效果，而且保留了比较好的并发性能。<br><br>
+ 事务的隔离性是通过锁实现，而事务的原子性、一致性和持久性则是通过事务日志实现。说到事务日志，不得不说的就是redo和undo。

#### Mysql怎么保证一致性的？
+ 从数据库层面
+ 数据库通过原子性、隔离性、持久性来保证一致性。也就是说ACID四大特性之中，C(一致性)是目的，A(原子性)、I(隔离性)、D(持久性)是手段，是为了保证一致性，数据库提供的手段。数据库必须要实现AID三大特性，才有可能实现一致性。例如，原子性无法保证，显然一致性也无法保证。<br><br>

+ 从应用层面，通过代码判断数据库数据是否有效，然后决定回滚还是提交数据！

##### Mysql怎么保证持久性的？
+ 1.redo log (保证持久性)
+ 在innoDB的存储引擎中，事务日志通过重做(redo)日志和innoDB存储引擎的日志缓冲(InnoDB Log Buffer)实现。Mysql是先把磁盘上的数据加载到内存中，在内存中对数据进行修改，再刷回磁盘上。事务开启时，事务中的操作，都会先写入存储引擎的日志缓冲中，在事务提交之前，这些缓冲的日志都需要提前刷新到磁盘上持久化，这就是DBA们口中常说的“日志先行”(Write-Ahead Logging)。当事务提交之后，在Buffer Pool中映射的数据文件才会慢慢刷新到磁盘。此时如果数据库崩溃或者宕机，那么当系统重启进行恢复时，就可以根据redo log中记录的日志，把数据库恢复到崩溃前的一个状态。未完成的事务，可以继续提交，也可以选择回滚，这基于恢复的策略而定。<br><br>

+ 在系统启动的时候，就已经为redo log分配了一块连续的存储空间,以顺序追加的方式记录Redo Log,通过顺序IO来改善性能。所有的事务共享redo log的存储空间，它们的Redo Log按语句的执行顺序，依次交替的记录在一起。如下一个简单示例：

+ 记录1：<trx1, insert...>

+ 记录2：<trx2, delete...>

+ 记录3：<trx3, update...>

+ 记录4：<trx1, update...>

+ 记录5：<trx3, insert...><br><br>

##### Mysql怎么保证原子性的？
+ 2.undo log (保证原子性)
+ undo log主要为事务的回滚服务。在事务执行的过程中，除了记录redo log，还会记录一定量的undo log。undo log记录了数据在每个操作前的状态，如果事务执行过程中需要回滚，就可以根据undo log进行回滚操作。当事务回滚时能够撤销所有已经成功执行的sql语句,单个事务的回滚，只会回滚当前事务做的操作，并不会影响到其他的事务做的操作。<br><br>

+ (1)当你delete一条数据的时候，就需要记录这条数据的信息，回滚的时候，insert这条旧数据
+ (2)当你update一条数据的时候，就需要记录之前的旧值，回滚的时候，根据旧值执行update操作
+ (3)当年insert一条数据的时候，就需要这条记录的主键，回滚的时候，根据主键执行delete操作<br><br>

+ undo log记录了这些回滚需要的信息，当事务执行失败或调用了rollback，导致事务需要回滚，便可以利用undo log中的信息将数据回滚到修改之前的样子。<br><br>

+ 以下是undo+redo事务的简化过程

+ 假设有2个数值，分别为A和B,值为1，2

+ 1. start transaction;
+ 2. 记录 A=1 到undo log;
+ 3. update A = 3；
+ 4. 记录 A=3 到redo log；
+ 5. 记录 B=2 到undo log；
+ 6. update B = 4；
+ 7. 记录B = 4 到redo log；
+ 8. 将redo log刷新到磁盘
+ 9. commit

+ 在1-8的任意一步系统宕机，事务未提交，该事务就不会对磁盘上的数据做任何影响。如果在8-9之间宕机，恢复之后可以选择回滚，也可以选择继续完成事务提交，因为此时redo log已经持久化。若在9之后系统宕机，内存映射中变更的数据还来不及刷回磁盘，那么系统恢复之后，可以根据redo log把数据刷回磁盘。

+ 所以，redo log其实保障的是事务的持久性和一致性，而undo log则保障了事务的原子性

#### 采用redo log的好处？

其实好处就是将redo log进行刷盘比对数据页刷盘效率高，具体表现如下

+ redo log体积小，毕竟只记录了哪一页修改了啥，因此体积小，刷盘快。
+ redo log是一直往末尾进行追加，属于顺序IO。效率显然比随机IO来的快。

#### Mysql怎么保证隔离性的
+ 锁
+ MVCC,即多版本并发控制(Multi Version Concurrency Control),一个行记录数据有多个版本对快照数据，这些快照数据在undo log中。 如果一个事务读取的行正在做DELELE或者UPDATE操作，读取操作不会等行上的锁释放，而是读取该行的快照版本。